In [49]:
from music21 import *
import numpy
import pandas as pd

In [3]:
#Display score of mxl file

score = converter.parse("Chopin__Trois_Valses.mxl")

# for m in b.getElementsByClass('Measure'):
#     k = m.analyze('key')
#     print(m.number, k)

# print(b)
#b.show(fmt = 'musicxml')    

In [65]:
score.show(fmt = 'musicxml')    

In [4]:

#find number of measures
beats = score.getTimeSignatures(recurse=False)[0].denominator

measures = int(score.highestTime / beats)
print(measures)
print(beats)

377
4


In [5]:
#chordify() makes score into only treble. (merges base into treble by making chords)
# Alternatively, can access the notes in each by doing score.part[index]

chords = score.chordify() 

In [6]:
length = chords.metronomeMarkBoundaries()[0][2].numberSounding #quarter notes per minute 
#Note, this up to offset 420.
qps = 60 / int(length) #quarter notes per second
print(length)
print(qps)

260
0.23076923076923078


In [10]:
chords.metronomeMarkBoundaries()

[(0.0, 420.0, <music21.tempo.MetronomeMark Quarter=260 (playback only)>),
 (420.0, 612.0, <music21.tempo.MetronomeMark Quarter=145 (playback only)>),
 (612.0, 709.0, <music21.tempo.MetronomeMark Quarter=130 (playback only)>),
 (709.0, 805.0, <music21.tempo.MetronomeMark Quarter=172 (playback only)>),
 (805.0, 900.0, <music21.tempo.MetronomeMark Quarter=145 (playback only)>),
 (900.0, 997.0, <music21.tempo.MetronomeMark Quarter=172 (playback only)>),
 (997.0, 1186.0, <music21.tempo.MetronomeMark Quarter=160 (playback only)>),
 (1186.0, 1442.0, <music21.tempo.MetronomeMark Quarter=145 (playback only)>),
 (1442.0, 1510.0, <music21.tempo.MetronomeMark Quarter=180 (playback only)>)]

In [61]:
#showing all of the notes in the score (1st column),
# along with their duration in beats (quarter note, eighth note, etc.),
# their duration in seconds (based on the BPM of the song),
# their frequency and their note name (should be directly correlated obviously),
# and the time in the song that they should be played (basically the sum of all the durations of the notes before).


note_cnt = 1

dur_beats_list = []
dur_seconds_list = []
freq_list = []
name_list = []
start_time_list = []

for el in chords.flatten():
    #print(el)
    if (type(el) == chord.Chord):
        for my_note in el:
            #print(note)
            note_cnt += 1
            name = my_note.nameWithOctave
            freq = my_note.pitch.frequency
            dur_seconds = my_note.duration.quarterLength * qps
            dur_beats = my_note.duration.type
            start_time = el.offset * qps #in seconds
            
            dur_beats_list.append(dur_beats)
            dur_seconds_list.append(dur_seconds)
            name_list.append(name)
            freq_list.append(freq)
            start_time_list.append(start_time)
            
            #print(freq) #offset is number of quarter notes?
#print(note_cnt)

In [62]:
dict_temp = {'Duration in Beats': dur_beats_list, 'Duration in Seconds': dur_seconds_list, 'Name': name_list, 'Frequency': freq_list, 'Start Time in Seconds': start_time_list}
df = pd.DataFrame(data=dict_temp)
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)
df

,Duration in Beats,Duration in Seconds,Name,Frequency,Start Time in Seconds
0,quarter,0.230769,A-4,415.304698,0.000000
1,eighth,0.115385,G4,391.995436,0.230769
2,eighth,0.115385,A-4,415.304698,0.346154
3,eighth,0.115385,C5,523.251131,0.461538
4,eighth,0.115385,B-4,466.163762,0.576923
...,...,...,...,...,...
7392,eighth,0.115385,A-4,415.304698,347.307692
7393,eighth,0.115385,E-4,311.126984,347.423077
7394,eighth,0.115385,A-3,207.652349,347.538462
7395,eighth,0.115385,E-3,155.563492,347.653846


In [120]:
score.flatten()

<music21.stream.Score 0x21d8a5f1050>

In [66]:
dur_beats_list = []
dur_seconds_list = []
freq_list = []
name_list = []
start_time_list = []


for measure in chords.getElementsByClass('Measure'):
    for n in measure.notesAndRests:
        if (type(n) == chord.Chord):
            for my_note in n:
                #print(note)
                name = my_note.nameWithOctave
                freq = my_note.pitch.frequency
                dur_seconds = my_note.duration.quarterLength * qps
                dur_beats = my_note.duration.type
                start_time = (measure.offset + n.offset) * qps #in seconds
                # note: This is calculated differently than above because the measure.offset gives the offset of the beginning of the measure.
                
                dur_beats_list.append(dur_beats)
                dur_seconds_list.append(dur_seconds)
                name_list.append(name)
                freq_list.append(freq)
                start_time_list.append(start_time)
        elif (type(n) == note.Rest):
            name = n.name
            freq = 0
            dur_seconds = n.duration.quarterLength * qps
            dur_beats = n.duration.type
            start_time = (measure.offset + n.offset) * qps
            
            dur_beats_list.append(dur_beats)
            dur_seconds_list.append(dur_seconds)
            name_list.append(name)
            freq_list.append(freq)
            start_time_list.append(start_time)
        else:
            print(type(n))
        #print(n)

In [67]:
dict_temp = {'Duration in Beats': dur_beats_list, 'Duration in Seconds': dur_seconds_list, 'Name': name_list, 'Frequency': freq_list, 'Start Time in Seconds': start_time_list}
df = pd.DataFrame(data=dict_temp)
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)
df

,Duration in Beats,Duration in Seconds,Name,Frequency,Start Time in Seconds
0,quarter,0.230769,A-4,415.304698,0.000000
1,eighth,0.115385,G4,391.995436,0.230769
2,eighth,0.115385,A-4,415.304698,0.346154
3,eighth,0.115385,C5,523.251131,0.461538
4,eighth,0.115385,B-4,466.163762,0.576923
...,...,...,...,...,...
7394,eighth,0.115385,A-4,415.304698,347.307692
7395,eighth,0.115385,E-4,311.126984,347.423077
7396,eighth,0.115385,A-3,207.652349,347.538462
7397,eighth,0.115385,E-3,155.563492,347.653846


In [11]:
# s = b
# sChords = s.chordify()
# sFlat = sChords.flatten()
# sOnlyChords = sFlat.getElementsByClass('Chord')
# displayPart = stream.Part(id='displayPart')
# for i in range(0, len(sOnlyChords) - 1):
#     thisChord = sOnlyChords[i]
#     nextChord = sOnlyChords[i + 1]
# for m in sChords.getElementsByClass('Measure'):
#     k = m.analyze('key')
#     print(m.number, k)

In [13]:
length = score.metronomeMarkBoundaries()[0][2].numberSounding
#length = 60 / int(length)
length

260